### Importing Libraries

In [1]:
import os

### Helper Function

In [2]:
os.getcwd()

'c:\\Users\\isaac\\Desktop\\GitHub\\Data-Science-Project\\3. Credit Card Fraud Detection\\datasets'

### Data Loading

### Data Visualization & Data Cleaning

### Feature Selection

### Preprocessing / Model Training